In [1]:
from NICU_offline.hobo import read_hobo_csv
import pandas
import glob
import re
import pyarrow.feather as feather
import numpy

# sites = ["01_02", "02_01", "04_01"]
# sites = ["02_01"]
sites = ["01_02"]
data_files = pandas.read_csv("data_files.csv")

for s in sites:
    resample = True
    site_data_sources = data_files.loc[data_files['site'] == s]
    site_hobo = list()
    # find all the eqs
    all_files = pandas.Series(dtype="string")
    for index, data_row in site_data_sources.iterrows():
        if data_row['read_hobo_files']:
            files = glob.glob(data_row['directory'] + data_row['hobo_file_prefix'] + "*.csv")
            all_files = pandas.concat([all_files, pandas.Series(files)])
    all_files = pandas.DataFrame({'filename': all_files, 'id': all_files})
    all_files['id'] = all_files['id'].str.replace(".*/", "")
    all_files['id'] = all_files['id'].str.replace("(^.*?)_.*", r"\1")
    all_ids = all_files['id'].unique()
    for id in all_ids:
        print("Consolidating {}".format(id))
        my_data = list()
        my_files = all_files.loc[all_files['id'] == id]
        for index, row in my_files.iterrows():
            hobo = read_hobo_csv(row['filename'])
            hobo = hobo.resample("60s").mean()
            my_data.append(hobo)
        my_data = pandas.concat(my_data)
        my_data = my_data.reset_index()
        new_row = pandas.DataFrame({'date':[numpy.nan], 'amps':[numpy.nan]})
        my_data = my_data.append(new_row)
        my_data.loc[my_data['amps'] > 5, 'amps'] = numpy.NaN
        if (id == "C-27"):
            begin_snip = pandas.Timestamp("2020-08-27 00:00:00", tz = "Asia/Kolkata")
            end_snip = pandas.Timestamp("2020-09-11 00:00:00", tz = "Asia/Kolkata")
            my_data.loc[numpy.logical_and(my_data.date > begin_snip, my_data.date < end_snip), 'amps'] = numpy.NaN
        if (id == "C-26"):
            begin_snip = pandas.Timestamp("2020-08-27 00:00:00", tz = "Asia/Kolkata")
            end_snip = pandas.Timestamp("2020-09-04 00:00:00", tz = "Asia/Kolkata")
            my_data.loc[numpy.logical_and(my_data.date > begin_snip, my_data.date < end_snip), 'amps'] = numpy.NaN
        if (id == "C-31"):
            begin_snip = pandas.Timestamp("2020-08-06 00:00:00", tz = "Asia/Kolkata")
            end_snip = pandas.Timestamp("2020-08-21 00:00:00", tz = "Asia/Kolkata")
            my_data.loc[numpy.logical_and(my_data.date > begin_snip, my_data.date < end_snip), 'amps'] = numpy.NaN
        if (id == "C-32"):
            begin_snip = pandas.Timestamp("2020-08-27 00:00:00", tz = "Asia/Kolkata")
            end_snip = pandas.Timestamp("2020-09-04 00:00:00", tz = "Asia/Kolkata")
            my_data.loc[numpy.logical_and(my_data.date > begin_snip, my_data.date < end_snip), 'amps'] = numpy.NaN
        if (id == "C-25"):
            begin_snip = pandas.Timestamp("2020-08-20 00:00:00", tz = "Asia/Kolkata")
            end_snip = pandas.Timestamp("2020-08-28 00:00:00", tz = "Asia/Kolkata")
            my_data.loc[numpy.logical_and(my_data.date > begin_snip, my_data.date < end_snip), 'amps'] = numpy.NaN
        if (id == "C-33"):
            begin_snip = pandas.Timestamp("2020-08-06 00:00:00", tz = "Asia/Kolkata")
            end_snip = pandas.Timestamp("2020-08-21 00:00:00", tz = "Asia/Kolkata")
            my_data.loc[numpy.logical_and(my_data.date > begin_snip, my_data.date < end_snip), 'amps'] = numpy.NaN
            begin_snip = pandas.Timestamp("2020-08-27 00:00:00", tz = "Asia/Kolkata")
            end_snip = pandas.Timestamp("2020-09-04 00:00:00", tz = "Asia/Kolkata")
            my_data.loc[numpy.logical_and(my_data.date > begin_snip, my_data.date < end_snip), 'amps'] = numpy.NaN
            begin_snip = pandas.Timestamp("2020-09-16 00:00:00", tz = "Asia/Kolkata")
            end_snip = pandas.Timestamp("2020-09-25 00:00:00", tz = "Asia/Kolkata")
            my_data.loc[numpy.logical_and(my_data.date > begin_snip, my_data.date < end_snip), 'amps'] = numpy.NaN
        data_filename = "data_cleaned_with_snips_and_current_exclusions/{}/{}.feather".format(s, id)
        feather.write_feather(my_data, data_filename)

Consolidating C-03
Consolidating C-02
Consolidating C-01
Consolidating C-04


In [2]:
hobo = read_hobo_csv(row['filename'])
hobo

,amps
date,
2020-09-20 11:23:56+05:30,1.0764
2020-09-20 11:24:06+05:30,0.8185
2020-09-20 11:24:16+05:30,0.6732
2020-09-20 11:24:26+05:30,0.4971
2020-09-20 11:24:36+05:30,0.4010
...,...
2020-10-03 11:54:46+05:30,0.4144
2020-10-03 11:54:56+05:30,0.3906
2020-10-03 11:55:06+05:30,0.3717


In [3]:
from NICU_offline.pqm import read_pqm_csv

for s in sites:
    site_data_sources = data_files.loc[data_files['site'] == s]
#     print(site_data_sources)
#     site_pqm_files = pandas.Series(dtype="string")
    site_pqm_data = list()
    for index, row in site_data_sources.iterrows():
        pqm_file = row['directory'] + row['pqm_file']
#         site_pqm_files = pandas.concat([site_pqm_files, pandas.Series(pqm_file)])
#         print(site_pqm_files)
#         if row['pqm_file'] != 'NO_POWER_FILE_PRESENT':
        if row['read_pqm_file']:
            pqm_data = read_pqm_csv(pqm_file)
            site_pqm_data.append(pqm_data)
#     for pqm_file in site_pqm_files:
    site_pqm_data = pandas.concat(site_pqm_data)
    site_pqm_data = site_pqm_data.reset_index()
#     site_pqm_data = site_pqm_data.sort_values(by=['date'])
#     site_pqm_data = site_pqm_data.drop_duplicates()
#     site_pqm_data = site_pqm_data.set_index('date')
    data_filename = "data_cleaned_with_snips_and_current_exclusions/{}/power.feather".format(s)
    feather.write_feather(site_pqm_data, data_filename)
#     data_filename = "data_cleaned/{}/power.csv".format(s)
#     site_pqm_data.to_csv(data_filename, index=False)       